In [1]:
import os
import sys
import datetime as dtm
import pandas as pd
import pyodbc

In [2]:
#// 2. Defining Classes (Classes definition)

class MSSQLdb:
    '''MSSQLdb Class: to manage interaction w/SQLSRV databases'''
    __cnt = 0
    catalog = {'Lista de conexiones a DB NO establecidas': [],
               'Lista de Errores de conexiones a DBs': [],
               'Lista de DBs Conectadas': []}
    
    def __init__(self, srv, db, usr=None, pwd=None, port=1433):
        pyodbc.pooling = False
        MSSQLdb.__cnt += 1
        self.id = MSSQLdb.__cnt
        self.drv = '{ODBC Driver 17 for SQL Server}'
        self.srv, self.port, self.db = srv, port, db
        self.usr, self.pwd = usr, pwd
        self.id_nm = f'''{self.id} _ * {self.srv} {self.db} *'''
        if usr and pwd:
            cnxstr = f'DRIVER={self.drv};SERVER={self.srv};PORT={self.port};\
                DATABASE={self.db};UID={self.usr};PWD={self.pwd}'
        else:
            cnxstr = f"DRIVER={self.drv};SERVER={self.srv};PORT={self.port};\
                DATABASE={self.db};Trusted_Connection=yes"
        self._cnx = pyodbc.connect(cnxstr)
        self._cur = self._cnx.cursor()
        self.closed = False

    def close(self):
        self._cur.close()
        self._cnx.close()
        self.closed = True
    
    def execute(self, sql, params=None):
        self._cur.execute(sql, params or ())
    
    def qry_to_df(self, sql, params=None):
        self.execute(sql, params)
        cols = [i[0] for i in self._cur.description]
        dats = [list(xx) for xx in self._cur]
        return pd.DataFrame(data=dats, columns=cols)

In [3]:
#// 3. Global Variables (Constants)
## Datetime Vars
now = dtm.datetime.now()
thisyr, thismnth = now.year, now.month      # To validate period red from list_tasks_file
now_fname = now.strftime('%Y%m%d%H%M')      # datetime firm for filenames

lst_fn = 'CSVs_list_fsd010.xlsx'
lst = 'listas/' + lst_fn

#// 4. Functions Definition
## Get f_corte (last day of period): AAAAMMDD  - last day of a month
def get_last_day_date(per):
    prd_as_date = dtm.datetime.strptime(per, '%Y%m').date()
    # day 25 exists in every month. 9 days later, it's always next month
    nxt_mnth = prd_as_date.replace(day=25) + dtm.timedelta(days=9)
    # subtracting the number of days of nxt_mnth we'll get last day of original month
    last_day_date = nxt_mnth - dtm.timedelta(days=nxt_mnth.day)
    return dtm.datetime.strftime(last_day_date, '%Y%m%d')

## Get f_corte_12: fecha de corte same month last year
def get_corte_last_year(per):
    prd_as_date = dtm.datetime.strptime(per, '%Y%m').date()
    prd_12_date = prd_as_date.replace(year= prd_as_date.year - 1)
    prd_12 = dtm.datetime.strftime(prd_12_date, '%Y%m')
    return get_last_day_date(prd_12)

#// 6. Read list_tasks_file and Get date values from period
## 6.1. Read list tasks_files: CSVs_list.xlsx
try:
    lst_df = pd.read_excel(lst)
except Exception as e:
    ln = f'''>ERROR en 6.1.! Falló la lectura de {lst}
        {e}'''
    print(ln)
    # wrt_info(ln, level='err_')
    # exit_prg()
else:
    # Display little sample of the result
    cols = ['Periodo', 'Carpeta', 'Nombre archivo CSV', 'Server Base']
    ln = f'''> {lst} leído correctamente:\n{lst_df[cols].head(3)}'''
    # wrt_info(ln, level='ok_')

## 6.2. Get period, f_corte and fcorte_12 as strings
try:
    period = str(int(lst_df.loc[0, 'Periodo']))

    ## Validate period: 6 chars
    assert len(period) == 6
    peryr, permnth = int(period[:4]), int(period[4:])
    # 4 first digs: year >= 2014 and <= actual year
    assert peryr >= 2013 and peryr <= thisyr
    # last 2 digs: month >= 1 and < actual if year==thisyear else 12
    assert permnth >= 1 and permnth < (thismnth if peryr == thisyr else 13)
except Exception as e:
    ln = f'''>ERROR en 5.2.! Periodo cargado: '{period}' INVALIDO!
            Períodos válidos: de 201401 a {thisyr}{thismnth-1}
            {e} '''
    # wrt_info(ln, level='err_')
    # exit_prg()
else:
    f_corte = "'" + get_last_day_date(period) + "'"
    fcorte_12 = f"'{get_corte_last_year(period)}'"
    ln = f'''> Fecha leída, Periodo: {period}.- Calculados; Fecha de corte: {f_corte},
            Corte del mismo mes del año anterior: {fcorte_12} '''
    # wrt_info(ln, level='ok_')

## 6.3 Get the total number of Files to be generated
total_files_to_process = len(lst_df)

print(lst)
print(f_corte, fcorte_12)
lst_df

listas/CSVs_list_fsd010.xlsx
'20230930' '20220930'


,Periodo,Carpeta,ix,Nombre archivo CSV,Server Base,Modo de armado,Disk,Path
0,202309,PDS,1,FSD010,172.31.119.50 Bantotal,"select Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aoo...",C,MTM_PCE_FSD010/CVS_periodo


In [4]:
#// 7. Replace period, f_corte, fcorte_12 in 'Path' and 'Modo de armado' of CVSs_list
lst_df['Path'] = lst_df['Path'].str.replace('periodo', period)
lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace('f_corte', f_corte)
lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace('fcorte_12', fcorte_12)
# replace strange "’" that appears w/libre office .xlsx
lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace("’", "'")

lst_df.loc[0, 'Modo de armado']

"select Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aooper,Aosbop,Aotope,convert(datetime,Aofval,121) as Aofval,convert(datetime,Aofvto,121) as Aofvto,convert(float,Aopzo) as Aopzo,Aottas,convert(float,Aotasa) as Aotasa,Aotmor,convert(float,Aottac) as Aottac,Aotasc,Aotdia,Aotvto,Aotano,Aotint,Aodrev,convert(float,Aoimp) as Aoimp,Aopre,Aopre1,Aotcbi,convert(float,Aotcbi1) as Aotcbi1,Aoarb,Aoarb1,Aomd,Aomd1,Aonume,Aofnum,Aoafiv,Aocbcu,Aostat,Aoavis,Aoplus,Aoeven,convert(datetime,Aofe99,121) as Aofe99,Aocltcod,Aoperiod from fsd010 where aomod in (101,102,103,104) and aopzo>0 and aofval <='20230930' union\nselect Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aooper,Aosbop,Aotope,convert(datetime,Aofval,121) as Aofval,convert(datetime,Aofvto,121) as Aofvto,convert(float,Aopzo) as Aopzo,Aottas,convert(float,Aotasa) as Aotasa,Aotmor,convert(float,Aottac) as Aottac,Aotasc,Aotdia,Aotvto,Aotano,Aotint,Aodrev,convert(float,Aoimp) as Aoimp,Aopre,Aopre1,Aotcbi,convert(float,Aotcbi1) as Aotcbi1,Aoarb,Aoarb1,Aomd,Aomd

In [5]:
try:                        # 1. connect to de DB - db_obj
    sb = lst_df.loc[0, 'Server Base'].split()
    srv, dbnm = sb[0], sb[1]
    srv_db = f'''{srv} {dbnm}'''
    ln = f''' DB_ Conectando con: {srv_db}...'''
    # wrt_info(ln)
    db = MSSQLdb(srv, dbnm, 'usr_motor', 'MTM.m0t0R2024')
except Exception as e:
    ln = f'''>DB_ERROR en 10.a! Falló la conexión con {srv_db}
            {e}'''
    MSSQLdb.catalog['Lista de Errores de conexiones a DBs'].append(ln)
    MSSQLdb.catalog['Lista de conexiones a DB NO establecidas'].append(srv_db )
    print(ln)
        # wrt_info(ln, level='err_')
        # continue
else:
    ln = f'''> DB_ Conexión con {srv} {dbnm} ESTABLECIDA! ({db.id_nm})'''
    MSSQLdb.catalog['Lista de DBs Conectadas'].append(db.id_nm)
    print(ln) 
        # wrt_info(ln, level='ok_')
    
total_rows = len(lst_df)
ln = f'Iniciando procesamiento de queries a {srv} {dbnm} - {total_rows} queries'
print(ln)

for row in range(total_rows):       # 2. loop in e/row of mini_df
    # 2.1. create cvs_obj - prev get vars from row
    folder = lst_df.loc[row, 'Carpeta']
    fn = lst_df.loc[row, 'Nombre archivo CSV']
    qry = lst_df.loc[row, 'Modo de armado']
    disk = lst_df.loc[row, 'Disk']
    path = lst_df.loc[row, 'Path']

df = db.qry_to_df(qry)

dic = df.dtypes.to_dict()
for col, typ in dic.items():
    if typ == 'datetime64[ns]':
        df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M:%S')

> DB_ Conexión con 172.31.119.50 Bantotal ESTABLECIDA! (1 _ * 172.31.119.50 Bantotal *)
Iniciando procesamiento de queries a 172.31.119.50 Bantotal - 1 queries


In [6]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 1000)
df

,Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aooper,Aosbop,Aotope,Aofval,Aofvto,Aopzo,Aottas,Aotasa,Aotmor,Aottac,Aotasc,Aotdia,Aotvto,Aotano,Aotint,Aodrev,Aoimp,Aopre,Aopre1,Aotcbi,Aotcbi1,Aoarb,Aoarb1,Aomd,Aomd1,Aonume,Aofnum,Aoafiv,Aocbcu,Aostat,Aoavis,Aoplus,Aoeven,Aofe99,Aocltcod,Aoperiod
0,1,101,10,80,0,106641,50288,0,1,2016-07-30 00:00:00,2021-08-01 00:00:00,1801.0,3,2.000,0.000000,0.0,0.000000,1,N,2,C,0,2757154.82,0E-8,0E-8,0E-8,10.0,0E-8,0E-8,,,2033420.90,1753-01-01 00:00:00,,0,99,0,0.000000,0,2021-08-03 00:00:00,0,30
1,1,101,10,80,0,107480,53528,0,1,2016-09-29 00:00:00,2019-11-20 00:00:00,1131.0,3,2.000,0.000000,0.0,0.000000,1,N,2,C,0,3270113.13,0E-8,0E-8,0E-8,10.0,0E-8,0E-8,,,1366667.81,1753-01-01 00:00:00,,0,99,0,0.000000,0,2018-03-26 00:00:00,0,30
2,1,101,10,80,0,107778,650033481,0,1,2013-07-23 00:00:00,2016-07-22 00:00:00,1079.0,3,3.750,0.000000,0.0,0.000000,1,N,2,C,0,175425.38,0E-8,0E-8,0E-8,1.0,0E-8,0E-8,,,149548.25,1753-01-01 00:00:00,,0,99,0,0.000000,0,2016-05-13 00:00:00,0,30
3,1,101,10,80,0,109085,56660,0,1,2016-11-24 00:00:00,2021-11-28 00:00:00,1804.0,3,2.000,0.000000,0.0,0.000000,1,N,2,C,0,379530.76,0E-8,0E-8,0E-8,10.0,0E-8,0E-8,,,284169.40,1753-01-01 00:00:00,,0,99,0,0.000000,0,2018-03-13 00:00:00,0,30
4,1,101,10,80,0,109922,660129213,0,1,1997-11-28 00:00:00,1999-05-13 00:00:00,525.0,3,2.501,0.000000,0.0,0.000000,1,N,2,C,0,24499.70,0E-8,0E-8,0E-8,1.0,0E-8,0E-8,,,6227.20,1753-01-01 00:00:00,,0,0,0,0.000000,0,1753-01-01 00:00:00,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189877,1,71,50,80,0,179239,577903,3,1,2012-04-25 00:00:00,2012-08-01 00:00:00,98.0,3,4.220,0.000000,0.0,0.000000,2,P,2,M,0,13979.21,0E-8,0E-8,0E-8,1.0,0E-8,0E-8,,,2020.79,1753-01-01 00:00:00,,0,99,0,0.000000,0,2017-12-29 00:00:00,0,0
189878,1,71,50,80,0,179239,583001,1,1,2012-07-18 00:00:00,2012-09-19 00:00:00,63.0,3,3.840,0.000000,0.0,0.000000,2,P,2,M,0,14320.71,0E-8,0E-8,0E-8,1.0,0E-8,0E-8,,,1179.29,1753-01-01 00:00:00,,0,0,0,0.000000,0,1753-01-01 00:00:00,0,0
189879,1,71,50,80,0,179239,583801,1,1,2012-08-03 00:00:00,2012-10-10 00:00:00,68.0,3,4.040,0.000000,0.0,0.000000,2,P,2,M,0,8227.20,0E-8,0E-8,0E-8,1.0,0E-8,0E-8,,,772.80,1753-01-01 00:00:00,,0,0,0,0.000000,0,1753-01-01 00:00:00,0,0
189880,1,71,50,80,0,325675,2269,1,1,2015-06-11 00:00:00,2015-07-10 00:00:00,29.0,3,2.000,0.000000,0.0,0.000000,2,P,2,M,0,71331.57,0E-8,0E-8,0E-8,1.0,0E-8,0E-8,,,1378.62,1753-01-01 00:00:00,,0,99,0,0.000000,0,2015-07-10 00:00:00,0,0


In [7]:
df.columns
df['Aofval']
df['Aofvto']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189882 entries, 0 to 189881
Data columns (total 42 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Pgcod     189882 non-null  int64  
 1   Aomod     189882 non-null  int64  
 2   Aosuc     189882 non-null  int64  
 3   Aomda     189882 non-null  int64  
 4   Aopap     189882 non-null  int64  
 5   Aocta     189882 non-null  int64  
 6   Aooper    189882 non-null  int64  
 7   Aosbop    189882 non-null  int64  
 8   Aotope    189882 non-null  int64  
 9   Aofval    189882 non-null  object 
 10  Aofvto    189882 non-null  object 
 11  Aopzo     189882 non-null  float64
 12  Aottas    189882 non-null  int64  
 13  Aotasa    189882 non-null  float64
 14  Aotmor    189882 non-null  object 
 15  Aottac    189882 non-null  float64
 16  Aotasc    189882 non-null  object 
 17  Aotdia    189882 non-null  int64  
 18  Aotvto    189882 non-null  object 
 19  Aotano    189882 non-null  int64  
 20  Aoti